# FORCE - use case testing
**FORCE (Framework for Operational Radiometric Correction for Environmental monitoring)**, an ‘all-in-one’ solution for the mass-processing and analysis of Landsat and Sentinel-2 image archives. FORCE is used to generate Level 2 ARD, and higher-level products, comprising state-of-the-art cloud masking and radiometric correction (including corrections that go beyond ARD specification, e.g., topographic or bidirectional reflectance distribution function correction).
Sara Aparicio

## 0 - Install libraries

In [2]:
from eo_utils import *
# from work/sample-notebooks/eo_utils import *
import numpy as np
import timeit

## 1 - Connect to backend & Authenticate
It is sent directly to VITO backend so the aggregator does not resolve the senidn of prior gap to correct tbakc that resolve beyond the point of collection. BOA S2 is a unique connection all 3 backends have been harmonized and the aggregator is not "smart" enough to resolve the backend. The way around is to select the backend:

#### 1.1 Select backend

In [12]:
# This would be the one choosed for the openEO Platform:
#backend = "openeo.cloud" 

# Instead we specify this one:
backend = "openeo-dev.eodc.eu/v1.0"

#### 1.2 Connect and authenticate

In [7]:
# connection = openeo.connect("openeo-dev.eodc.eu")
conn = openeo.connect(backend).authenticate_oidc(provider_id="egi")

Authenticated using refresh token.


#### 1.3 Sets the map:
Note that you need the helper script eo_utils.py in order to run this cell!

In [9]:
center = [46.49, 11.35]
zoom = 12

eoMap = openeoMap(center,zoom)
#addS2Tiles(eoMap)
eoMap.map

Map(center=[46.49, 11.35], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [10]:
# bbox = [3.236323, 43.474184 , 16.198021, 45.818592]
# bbox = eoMap.getBbox()
# bbox = [11.210517883300781, 46.44258864468262, 11.489639282226564, 46.53713734839792]
# west 3.236323, # east 16.198021, # south 43.474184 
# north 45.818592

# west 7.739853 
# east 14.751275 
# south 50.541363 
# north 53.120405

# west 9.256786 
# east 14.970855 
# south 44.505908 
# north 47.526104

bbox = [9.256786, 44.505908, 14.970855, 47.526104]
print('west',bbox[0],'\neast',bbox[2],'\nsouth',bbox[1],'\nnorth',bbox[3])

west 9.256786 
east 14.970855 
south 44.505908 
north 47.526104


In [11]:
bbox = eoMap.getBbox()
print(bbox)

[10.789947509765627, 46.38578052681784, 12.63427734375, 46.76338378225934]


## 2- Selecting constraints for the S2 collection (cube)
Here the dates selected are on porpous not older than 3 months, since datasets older than 3 months (i.e. from current date) are stored and hence not available. As such, when using a 3-month old dataset it has to be recalled and there is currenyly an issue with this recall.

In [13]:
collection      = "SENTINEL2_L1C"#'sentinel-2-l1c'#'SENTINEL2_L1C'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':'EPSG:4326'}
temporal_extent = ["2023-10-09", "2023-10-19"]
bands           = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11']

#### 2.1 Creating the S2 cube
Natively it cannot run again, because of the harmonization collection error. However, if setting the correct properties as described in https://github.com/openEOPlatform/architecture-docs/issues/357 by Valentina, this way a specific backend in provided and should run this way.

In [14]:
cube = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent, properties={"provider:backend":lambda v: v =="eodc"})

#### 2.2 Applying FORCE to the S2 Cube

In [16]:
cube = cube.atmospheric_correction(method="FORCE")
cube = cube.save_result(format='GTiff')

#### 2.3 Create the job

In [17]:
job = cube.create_job()
job.start_job()

Preflight process graph validation raised: [404] Load call not available for SENTINEL2_L1C


<BatchJob job_id='c4d7178a-f7b5-4ba9-a157-7c4938f022c1'>

In [8]:
ard_force = cube.ard_surface_reflectance(atmospheric_correction_method = 'FORCE', 
                                       cloud_detection_method = 'Fmask', 
                                       elevation_model = "cop-dem-30m",
                                       atmospheric_correction_options = {"do_brdf" : "TRUE", "do_topo" : "TRUE"}, 
                                       cloud_detection_options={'cld_prob' : 0.225,'cld_dil' : 6, 'shd_dil' : 6})

In [9]:
ard_force_tif = ard_force.save_result(format="GTiff")

In [ ]:
# job = ard_force_tif.execute_batch(format = 'GTiff')
# job.get_results().download_files("sar-nrb")

In [11]:
job_ard_force_tif = connection.create_job(ard_force_tif)#.graph
job_ard_force_tif.start_job()

<BatchJob job_id='eodc-e25ca8bf-47dd-44b4-a401-826dc836bceb'>

In [15]:
ard_force_tif.graph

AttributeError: 'DataCube' object has no attribute 'graph'

In [16]:
#job_ard_force_tif = connection.create_job(ard_force_tif.graph)
job_id_ard_force_tif = job_ard_force_tif.job_id
if job_id_ard_force_tif:
    print("Batch job created with id: ",job_id_ard_force_tif)
    job_ard_force_tif.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  eodc-e25ca8bf-47dd-44b4-a401-826dc836bceb


In [17]:
job_ard = connection.job(job_id_ard_force_tif)
job_description_ard = job_ard.describe_job()
print("Batch job with id: ",job_id_ard_force_tif, ' is ',job_description_ard['status'])

Batch job with id:  eodc-e25ca8bf-47dd-44b4-a401-826dc836bceb  is  running


In [ ]:
print("Batch job with id: ",job_id_ard_force_tif, ' is ',job_ard.describe_job()['status'])

In [ ]:
job_description_ard

In [ ]:
results_ard = job_ard.get_results()
# metadata_ard = results_ard.get_metadata()

In [ ]:
results_ard.download_files("./data/ARD_FORCE_03-2022")
# results_ard.download("./data/ARD_FORCE_03-2022",format="GTiff")

In [ ]:
openeo.__version__